In [ ]:
pip install yfinance

In [48]:
import numpy as np
import pandas as pd
import datetime
import yfinance as yf

### Tesla Stock Price, NASDAQ Composite, S&P 500 and Oil price

In [196]:
tesla_idx = "TSLA"
NASDAQ_composite_idx = '^IXIC'
SnP_500_idx = '^GSPC'
oil_price_idx = 'AAPL'

start_date = "2021-01-01"
end_date = "2022-12-30" # Il n'y a pas les données pour le 31 décembre 2022, le 01 et 02 janvier 2023

tesla_stock_price = yf.download(tesla_idx, start=start_date, end=end_date)
NASDAQ_composite = yf.download(NASDAQ_composite_idx, start=start_date, end=end_date)
SnP_500 = yf.download(SnP_500_idx, start=start_date, end=end_date)
oil_price = yf.download(oil_price_idx, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [197]:
# Création d'un array avec toutes les dates du 3 janvier 2021 au 4 janvier 2023
date_debut = datetime.datetime(2021, 1, 1)
date_fin = datetime.datetime(2022, 12, 30)
dates = np.arange(date_debut, date_fin, datetime.timedelta(days=1))

# Récupération des jours d'ouvertures du marché
market_days = tesla_stock_price.index.to_numpy()

# Récupération des valeurs en bourses à l'ouverture de nos paramètres
tesla_open = tesla_stock_price['Open'].to_numpy()
NASDAQ_open = NASDAQ_composite['Open'].to_numpy()
SnP_500_open = SnP_500['Open'].to_numpy()
oil_open = oil_price['Open'].to_numpy()


### Tesla deaths cases

In [198]:
df_deaths = pd.read_excel("../Datasets/Tesla Deaths.xlsx", header = 1)

deaths_dates = df_deaths['Date'].to_numpy()
deaths_count = df_deaths['Deaths'].to_numpy(dtype=int)


### Tweets of Elon Musk

In [199]:
df_tweets = pd.read_csv("../Datasets/Elon musk tweets 2022.csv")

tweets_dates = np.array([np.datetime64(date.split(' ')[0]) for date in df_tweets['date'].to_numpy()], dtype='datetime64[us]')
tweets_content = df_tweets['Content'].to_numpy()
tweets_likes = df_tweets['Likes'].to_numpy()
tweets_retweet = df_tweets['Retweet'].to_numpy()


### Mise en forme de toutes les colonnes en format np.array

In [331]:
# Tous les lags qui seront utilisés dans le DataFrame final
lag1 = 2
lag3 = 4
lag7 = 8
lag30 = 31
nb_days = len(dates)

# Voici la liste de tous les paramètres
parametres = ['Date', 'Tesla Stock Price', 'NASDAQ Composite', 'S&P 500', 'Oil Price', 
              'Deaths', 'Deaths Lag '+str(lag1), 'Deaths Lag '+str(lag7), 'Deaths Lag '+str(lag30), 
              'Tweets of Elon Musk', 'Tweet with mention Tesla','% Tweet w/ mention Tesla', 
              'Tweet w/ mention Tesla Lag '+str(lag1), 'Tweet w/ mention Tesla Lag '+str(lag3), 
              'Tweet w/ mention Tesla Lag '+str(lag7), 'Tweet Likes', 'Tweet Likes Lag '+str(lag1),
              'Tweet Likes Lag '+str(lag3), 'Tweet Likes Lag '+str(lag7), 'Retweet', 
              'Retweet Lag '+str(lag1), 'Retweet Lag '+str(lag3),  'Retweet Lag '+str(lag7)]

# data[i,0] = np.datetime_as_string(dates[i], unit='D')

In [219]:
len(parametres)

23

#### Variable 1 : Dates

In [224]:
dates_format_df = np.array([np.datetime_as_string(dates[i], unit='D') for i in range(nb_days)])

In [226]:
dates_format_df[0]

'2021-01-03'

#### Variables 1 à 5 : Date, Tesla Stock, NASDAQ Composite, S&P 500, Prix du pétrole

In [322]:
market_nb_para = 5

data_market = np.zeros([nb_days,market_nb_para], dtype = object)
count_market_day = 0 # Cette variable nous permet de suivre les jours d'ouverture du marché

for i in range(nb_days):
    
    #Ajout dates
    data_market[i,0] = dates_format_df[i]
    # Ajout valeurs sur le marché
    data_market[i,1] = tesla_open[count_market_day]
    data_market[i,2] = NASDAQ_open[count_market_day]
    data_market[i,3] = SnP_500_open[count_market_day]
    data_market[i,4] = oil_open[count_market_day]
    # On suit les jours d'ouverture du marché
    if dates[i] == market_days[count_market_day]:
        count_market_day += 1

In [323]:
data_market[10]

array(['2021-01-13', 284.2533264160156, 13088.009765625, 3802.22998046875,
       128.75999450683594], dtype=object)

#### Variables 6 et 7 : Nombre de morts, avec lag de 1 jour, 7 jours et 30 jours

In [327]:
deaths_nb_para = 4

data_deaths = np.zeros([nb_days,deaths_nb_para], dtype = int)

for i in range(nb_days):
    
    # Ajout des morts, on cherche s'il y a eu un déces à la dates[i] grâce à notre liste des dates de décès
    idx_death = np.where(deaths_dates == dates[i])[0]
    if len(idx_death) > 0:
        for j in idx_death:
            # Mort sans lag
            data_deaths[i,0] = deaths_count[j]
            
            # Mort avec lag1
            if nb_days-i >= lag1:
                data_deaths[i:i+lag1,1] += deaths_count[j]
            else:
                data_deaths[i:,1] += deaths_count[j]
                
            # Mort avec lag7
            if nb_days-i >= lag7:
                data_deaths[i:i+lag7,2] += deaths_count[j]
            else:
                data_deaths[i:,2] += deaths_count[j]
                
            # Mort avec lag30
            if nb_days-i >= lag30:
                data_deaths[i:i+lag30,3] += deaths_count[j]
            else:
                data_deaths[i:,3] += deaths_count[j]

In [253]:
data_deaths[12]

array([1, 1, 3, 3])

#### Variables 8 à 13 : Nombre de tweets; Nombre avec mention Tesla; Proportion avec mention Tesla; Tweets Tesla lag 1 jour, 3 jours, 1 semaine; Nombre de likes, avec/sans lags; Nombre de retweets, avec/sans lags

In [274]:
tweets_nb_para = 14

data_tweets = np.zeros([nb_days,tweets_nb_para])

for i in range(nb_days):
    
    # Ajout de la date des tweets, likes et retweet, ainsi que la mention du mot Tesla
    idx_tweet = np.where(tweets_dates == dates[i])[0]
    nb_tweets = len(idx_tweet)
    if nb_tweets > 0:
        # On ajoute le nombre de tweets
        data_tweets[i,0] = nb_tweets
        
        for j in idx_tweet:
            # On regarde si le tweet contient le mot tesla
            if 'Tesla' in tweets_content[j] or 'tesla' in tweets_content[j]:
                data_tweets[i,1] += 1
                # On ajoute le nombre de tweets avec lag1
                if nb_days-i >= lag1:
                    data_tweets[i:i+lag1,3] += 1
                else:
                    data_tweets[i:,3] += 1
                # On ajoute le nombre de tweets avec lag3
                if nb_days-i >= lag3:
                    data_tweets[i:i+lag3,4] += 1
                else:
                    data_tweets[i:,4] += 1
                # On ajoute le nombre de tweets avec lag30
                if nb_days-i >= lag7:
                    data_tweets[i:i+lag30,5] += 1
                else:
                    data_tweets[i:,5] += 1
            
            # Nombre de likes
            data_tweets[i,6] += tweets_likes[j]
            if nb_days-i >= lag1:
                data_tweets[i:i+lag1,7] += 1
            else:
                data_tweets[i:,7] += 1
            # On ajoute le nombre de tweets avec lag3
            if nb_days-i >= lag3:
                data_tweets[i:i+lag3,8] += 1
            else:
                data_tweets[i:,8] += 1
            # On ajoute le nombre de tweets avec lag30
            if nb_days-i >= lag7:
                data_tweets[i:i+lag30,9] += 1
            else:
                data_tweets[i:,9] += 1
            
            # Nombre de retweet
            data[i,10] += tweets_retweet[j]
            if nb_days-i >= lag1:
                data_tweets[i:i+lag1,11] += 1
            else:
                data_tweets[i:,11] += 1
            # On ajoute le nombre de tweets avec lag3
            if nb_days-i >= lag3:
                data_tweets[i:i+lag3,12] += 1
            else:
                data_tweets[i:,12] += 1
            # On ajoute le nombre de tweets avec lag30
            if nb_days-i >= lag7:
                data_tweets[i:i+lag30,13] += 1
            else:
                data_tweets[i:,13] += 1
        
        # On ajoute la proportions de mention Tesla
        data[i,2] = data[i,1]/data[i,0]

In [325]:
data_tweets[-8]

array([2.000000e+01, 2.000000e+00, 0.000000e+00, 3.000000e+00,
       3.000000e+00, 3.200000e+01, 2.081515e+06, 6.700000e+01,
       1.140000e+02, 1.007000e+03, 0.000000e+00, 6.700000e+01,
       1.140000e+02, 1.007000e+03])

### Mise en DataFrame

In [332]:
all_data = np.concatenate((data_market, data_deaths, data_tweets), axis = 1)

df = pd.DataFrame (all_data, columns = parametres)

In [333]:
df.tail(10)

,Date,Tesla Stock Price,NASDAQ Composite,S&P 500,Oil Price,Deaths,Deaths Lag 2,Deaths Lag 8,Deaths Lag 31,Tweets of Elon Musk,...,Tweet w/ mention Tesla Lag 4,Tweet w/ mention Tesla Lag 8,Tweet Likes,Tweet Likes Lag 2,Tweet Likes Lag 4,Tweet Likes Lag 8,Retweet,Retweet Lag 2,Retweet Lag 4,Retweet Lag 8
716,2022-12-20,146.050003,10490.889648,3810.469971,131.389999,0,1,3,10,39.0,...,3.0,35.0,2563494.0,47.0,137.0,1015.0,0.0,47.0,137.0,1015.0
717,2022-12-21,139.339996,10592.0,3839.48999,132.979996,0,0,3,10,47.0,...,3.0,33.0,1907404.0,86.0,140.0,1027.0,0.0,86.0,140.0,1027.0
718,2022-12-22,136.0,10586.459961,3853.26001,134.350006,1,1,4,11,20.0,...,3.0,32.0,2081515.0,67.0,114.0,1007.0,0.0,67.0,114.0,1007.0
719,2022-12-23,126.370003,10437.75,3815.110107,130.919998,0,1,4,11,35.0,...,4.0,31.0,1880824.0,55.0,141.0,1013.0,0.0,55.0,141.0,1013.0
720,2022-12-24,117.5,10462.19043,3843.340088,131.380005,0,0,4,11,22.0,...,4.0,31.0,948729.0,57.0,124.0,966.0,0.0,57.0,124.0,966.0
721,2022-12-25,117.5,10462.19043,3843.340088,131.380005,0,0,3,11,6.0,...,3.0,30.0,629163.0,28.0,83.0,943.0,0.0,28.0,83.0,943.0
722,2022-12-26,117.5,10462.19043,3843.340088,131.380005,0,0,2,11,19.0,...,1.0,30.0,707518.0,25.0,82.0,935.0,0.0,25.0,82.0,935.0
723,2022-12-27,117.5,10462.19043,3843.340088,131.380005,0,0,1,10,20.0,...,0.0,29.0,1796721.0,39.0,67.0,931.0,0.0,39.0,67.0,931.0
724,2022-12-28,110.349998,10339.200195,3829.560059,129.669998,0,0,1,9,49.0,...,0.0,25.0,4565581.0,69.0,94.0,948.0,0.0,69.0,94.0,948.0
725,2022-12-29,120.389999,10321.459961,3805.449951,127.989998,0,0,1,7,9.0,...,1.0,25.0,576685.0,58.0,97.0,908.0,0.0,58.0,97.0,908.0


In [154]:
df.to_excel('Data.xlsx', index = False)